# Modelo de Machine Learning

In [35]:
import numpy as np
import pandas as pd
import os
import math
from random import randint

# Ruta de archivos

In [101]:
path_train = os.path.join(os.getcwd(), '..', '4_modeling_module',"TRAIN_DATA")
path_dwh = os.path.join(os.getcwd(), '..', '3_data_preprocesing_module',"DATAWAREHOUSE")
path_scoring_db = os.path.join(os.getcwd(),"DATABASE_SCORING")

In [76]:
df_razon_social = pd.read_csv(os.path.join(path_dwh,'dim_razon_social.csv'))
df_fecha = pd.read_csv(os.path.join(path_dwh,'dim_fecha.csv'))
df_curso_perfil = pd.read_csv(os.path.join(path_dwh,'dim_curso_perfil.csv'))

In [149]:
df_ci_cp = pd.read_csv(os.path.join(path_dwh,'fact_ci_cp.csv'))
df_ci_rs = pd.read_csv(os.path.join(path_dwh,'fact_ci_rs.csv'))
df_oc_cp = pd.read_csv(os.path.join(path_dwh,'fact_oc_cp.csv'))
df_oc_rs = pd.read_csv(os.path.join(path_dwh,'fact_oc_rs.csv'))
df_oec_cp = pd.read_csv(os.path.join(path_dwh,'fact_oec_cp.csv'))
df_oec_rs = pd.read_csv(os.path.join(path_dwh,'fact_oec_rs.csv'))

In [26]:
df_fecha = pd.read_csv(os.path.join(path_scoring,'dim_fecha.csv'))

In [27]:
df_fecha.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2142 entries, 0 to 2141
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id_fecha  2142 non-null   int64 
 1   fecha     2142 non-null   object
 2   dia       2142 non-null   int64 
 3   mes       2142 non-null   int64 
 4   anio      2142 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 83.8+ KB


In [29]:
df_fecha.tail()

,id_fecha,fecha,dia,mes,anio
2137,2137,31-10-2017,31,10,2017
2138,2138,31-10-2018,31,10,2018
2139,2139,31-10-2019,31,10,2019
2140,2140,31-10-2020,31,10,2020
2141,2141,31-10-2021,31,10,2021


# Generación de datos ficticios

Generación de nuevas fechas

In [90]:
nueva_fecha = [
    [2142,'31-01-2023',31,1,2023],
    [2143,'28-02-2023',28,2,2023]
]

In [91]:
df_new_fecha = pd.DataFrame(data=nueva_fecha,columns=df_fecha.columns)

In [92]:
df_fecha = pd.concat([df_fecha,df_new_fecha],ignore_index=True)

## CI CP

In [119]:

new_data_ci_cp = [] 
size_ci_cp = df_ci_cp.shape[0]
size_new_fecha =df_new_fecha.shape[0]
for i in range(300):
    rd_idx = randint(0,size_ci_cp-1)
    rd_f_idx = randint(0,size_new_fecha-1)
    id_rs_rd = df_ci_cp['id_razon_social'].iloc[rd_idx]
    id_cp_rd = df_ci_cp['id_curso_perfil'].iloc[rd_idx]
    id_fecha_rd = df_new_fecha['id_fecha'].iloc[rd_f_idx]
    rd_num_cap = 0
    rd_total_horas = 0
    rd_total_ganancias = 0
    new_data_ci_cp.append([id_rs_rd,id_cp_rd,id_fecha_rd,rd_num_cap,rd_total_horas,rd_total_ganancias])
    
   
df_new_ci_cp = pd.DataFrame(data=new_data_ci_cp,columns=['id_razon_social',	'id_curso_perfil',	'id_fecha',	'num_cap',	'total_horas',	'total_ganancias'])
    

In [120]:
df_new_ci_cp = df_new_ci_cp.drop_duplicates()

In [121]:
df_new_ci_cp['id_fact_ci_cp'] = [x for x in range(df_ci_cp.shape[0],df_ci_cp.shape[0]+df_new_ci_cp.shape[0])]

In [122]:
df_new_ci_cp.insert(0, 'id_fact_ci_cp', df_new_ci_cp.pop('id_fact_ci_cp'))

In [132]:
df_ci_cp = pd.concat([df_ci_cp,df_new_ci_cp],ignore_index=True)

In [125]:
data_ci_cp = pd.merge(data_ci_cp,df_curso_perfil,on='id_curso_perfil',how='inner')
data_ci_cp = pd.merge(data_ci_cp,df_fecha,on='id_fecha',how='inner')
data_ci_cp = pd.merge(data_ci_cp,df_razon_social,on='id_razon_social',how='inner')

In [126]:
data_ci_cp = data_ci_cp.drop(['num_cap','total_horas','total_ganancias'],axis=1)

In [128]:
data_ci_cp.to_csv(os.path.join(path_scoring_db,'data_ci_cp_sc.csv'),index=False)

## CI RS

In [158]:
df_new_ci_rs = df_new_ci_cp.groupby(['id_razon_social','id_fecha']).count().reset_index()[['id_razon_social','id_fecha']]

In [159]:
df_new_ci_rs['id_fact_ci_rs'] = [x for x in range(df_ci_rs.shape[0],df_ci_rs.shape[0]+df_new_ci_rs.shape[0])]
df_new_ci_rs.insert(0, 'id_fact_ci_rs', df_new_ci_rs.pop('id_fact_ci_rs'))
data_ci_rs = pd.concat([df_ci_rs,df_new_ci_rs],ignore_index=True)

In [160]:
data_ci_rs = data_ci_rs.drop(['volumen_cap','volumen_horas','volumen_ganancias','total_cursos'],axis=1)

In [162]:
data_ci_rs = pd.merge(data_ci_rs,df_fecha,on='id_fecha',how='inner')
data_ci_rs = pd.merge(data_ci_rs,df_razon_social,on='id_razon_social',how='inner')

In [176]:
data_ci_rs.to_csv(os.path.join(path_scoring_db,'data_ci_rs_sc.csv'),index=False)

## OC CP 

In [189]:
new_data_oc_cp = [] 
size_oc_cp = df_oc_cp.shape[0]
for i in range(300):
    rd_idx = randint(0,size_ci_cp-1)
    rd_f_idx = randint(0,size_new_fecha-1)
    id_rs_rd = df_oc_cp['id_razon_social'].iloc[rd_idx]
    id_cp_rd = df_oc_cp['id_curso_perfil'].iloc[rd_idx]
    id_fecha_rd = df_new_fecha['id_fecha'].iloc[rd_f_idx]
    new_data_oc_cp.append([id_rs_rd,id_cp_rd,id_fecha_rd])
    
   
df_new_oc_cp = pd.DataFrame(data=new_data_oc_cp,columns=['id_razon_social',	'id_curso_perfil',	'id_fecha'])
    

In [190]:
df_new_oc_cp = df_new_oc_cp.drop_duplicates()
df_new_oc_cp['id_fact_oc_cp'] = [x for x in range(df_oc_cp.shape[0],df_oc_cp.shape[0]+df_new_oc_cp.shape[0])]
df_new_oc_cp.insert(0, 'id_fact_oc_cp', df_new_oc_cp.pop('id_fact_oc_cp'))
data_oc_cp = pd.concat([df_oc_cp,df_new_oc_cp],ignore_index=True)

In [191]:
data_oc_cp.dropna(axis='columns',inplace=True)

In [192]:
data_oc_cp = pd.merge(data_oc_cp,df_curso_perfil,on='id_curso_perfil',how='inner')
data_oc_cp = pd.merge(data_oc_cp,df_fecha,on='id_fecha',how='inner')
data_oc_cp = pd.merge(data_oc_cp,df_razon_social,on='id_razon_social',how='inner')

In [193]:
data_oc_cp.to_csv(os.path.join(path_scoring_db,'data_oc_cp_sc.csv'),index=False)

## OC RS

In [233]:
df_new_oc_rs = df_new_oc_cp.groupby(['id_razon_social','id_fecha']).count().reset_index()
df_new_oc_rs['id_fact_oc_rs'] = [x for x in range(df_oc_rs.shape[0],df_oc_rs.shape[0]+df_new_oc_rs.shape[0])]
df_new_oc_rs.insert(0, 'id_fact_oc_rs', df_new_oc_rs.pop('id_fact_oc_rs'))
data_oc_rs = pd.concat([df_oc_rs,df_new_oc_rs],ignore_index=True)
data_oc_rs.dropna(axis='columns',inplace=True)
data_oc_rs = pd.merge(data_oc_rs,df_fecha,on='id_fecha',how='inner')
data_oc_rs = pd.merge(data_oc_rs,df_razon_social,on='id_razon_social',how='inner')


In [235]:
data_oc_rs.shape

(396, 13)

In [234]:
data_oc_rs.to_csv(os.path.join(path_scoring_db,'data_oc_rs_sc.csv'),index=False)

## OEC CP

In [224]:
new_data_oec_cp = [] 
size_oec_cp = df_oec_cp.shape[0]
for i in range(300):
    rd_idx = randint(0,size_ci_cp-1)
    rd_f_idx = randint(0,size_new_fecha-1)
    id_rs_rd = df_oec_cp['id_razon_social'].iloc[rd_idx]
    id_cp_rd = df_oec_cp['id_curso_perfil'].iloc[rd_idx]
    id_fecha_rd = df_new_fecha['id_fecha'].iloc[rd_f_idx]
    new_data_oec_cp.append([id_rs_rd,id_cp_rd,id_fecha_rd])
    
   
df_new_oec_cp = pd.DataFrame(data=new_data_oec_cp,columns=['id_razon_social',	'id_curso_perfil',	'id_fecha'])
    

In [225]:
df_new_oec_cp = df_new_oec_cp.drop_duplicates()
df_new_oec_cp['id_fact_oec_cp'] = [x for x in range(df_oec_cp.shape[0],df_oec_cp.shape[0]+df_new_oec_cp.shape[0])]
df_new_oec_cp.insert(0, 'id_fact_oec_cp', df_new_oec_cp.pop('id_fact_oec_cp'))
data_oec_cp = pd.concat([df_oec_cp,df_new_oec_cp],ignore_index=True)
data_oec_cp.dropna(axis='columns',inplace=True)
data_oec_cp = pd.merge(data_oec_cp,df_curso_perfil,on='id_curso_perfil',how='inner')
data_oec_cp = pd.merge(data_oec_cp,df_fecha,on='id_fecha',how='inner')
data_oec_cp = pd.merge(data_oec_cp,df_razon_social,on='id_razon_social',how='inner')

In [226]:
data_oec_cp.shape

(18399, 21)

In [227]:
data_oec_cp.to_csv(os.path.join(path_scoring_db,'data_oec_cp_sc.csv'),index=False)

## OEC RS

In [229]:
df_new_oec_rs = df_new_oec_cp.groupby(['id_razon_social','id_fecha']).count().reset_index()
df_new_oec_rs['id_fact_oec_rs'] = [x for x in range(df_oec_rs.shape[0],df_oec_rs.shape[0]+df_new_oec_rs.shape[0])]
df_new_oec_rs.insert(0, 'id_fact_oec_rs', df_new_oec_rs.pop('id_fact_oec_rs'))
data_oec_rs = pd.concat([df_oec_rs,df_new_oec_rs],ignore_index=True)
data_oec_rs.dropna(axis='columns',inplace=True)
data_oec_rs = pd.merge(data_oec_rs,df_fecha,on='id_fecha',how='inner')
data_oec_rs = pd.merge(data_oec_rs,df_razon_social,on='id_razon_social',how='inner')


In [232]:
data_oec_rs.to_csv(os.path.join(path_scoring_db,'data_oec_rs_sc.csv'),index=False)

r

In [ ]:
df = []
fact_oc_cp = None
fact_oec_cp = None
fact_ci_cp = None
fact_oc_rs = None
fact_oec_rs = None
fact_ci_rs = None
for file in filelist:
    if '.csv' in file:
            
        df = pd.read_csv(os.path.join(path_model,file))
        idx_pred = [ i for i, word in enumerate(df.columns) if word.startswith('prediction') ]
        pred_col = df.columns[idx_pred][0]
        if 'oc_cp_' in file:
            if fact_oc_cp is None:
                fact_oc_cp = df
            else: 
                fact_oc_cp = fact_oc_cp.merge(df[['id_fact_oc_cp',pred_col]],on='id_fact_oc_cp',how='inner')

        elif 'oec_cp_' in file:
            if fact_oec_cp is None:
                fact_oec_cp = df
            else:
                fact_oec_cp = fact_oec_cp.merge(df[['id_fact_oec_cp',pred_col]],on='id_fact_oec_cp',how='inner')

        elif 'ci_cp_' in file:
            if fact_ci_cp is None:
                fact_ci_cp = df
            else:
                fact_ci_cp = fact_ci_cp.merge(df[['id_fact_ci_cp',pred_col]],on='id_fact_ci_cp',how='inner')

        elif 'oc_rs_' in file:
            if fact_oc_rs is None:
                fact_oc_rs = df
            else:
                fact_oc_rs = fact_oc_rs.merge(df[['id_fact_oc_rs',pred_col]],on='id_fact_oc_rs',how='inner')

        elif 'oec_rs_' in file:
            if fact_oec_rs is None:
                fact_oec_rs = df
            else:
                fact_oec_rs = fact_oec_rs.merge(df[['id_fact_oec_rs',pred_col]],on='id_fact_oec_rs',how='inner')

        elif 'ci_rs_' in file:
            if fact_ci_rs is None:
                fact_ci_rs = df
            else:
                fact_ci_rs = fact_ci_rs.merge(df[['id_fact_ci_rs',pred_col]],on='id_fact_ci_rs',how='inner')
